# 8 schools data

In [1]:
from numpyro.infer import Predictive
from numpyro.infer.reparam import TransformReparam, LocScaleReparam
from jax import random
from numpyro.infer import MCMC, HMC
import numpyro.distributions as dist
import numpyro
import numpy as np

Let us explore NumPyro using a simple example. We will use the eight schools example from Gelman et al., Bayesian Data Analysis: Sec. 5.5, 2003, which studies the effect of coaching on SAT performance in eight schools.

The data is given by:

In [2]:
J = 8
y = np.array([28.0, 8.0, -3.0, 7.0, -1.0, 1.0, 18.0, 12.0])
sigma = np.array([15.0, 10.0, 16.0, 11.0, 9.0, 11.0, 10.0, 18.0])

where `y` are the treatment effects and `sigma` the standard error. 

We build a hierarchical model for the study where we assume that the group-level parameters `theta` for each school are sampled from a Normal distribution with unknown mean `mu` and standard deviation `tau`, while the observed data are in turn generated from a Normal distribution with mean and standard deviation given by `theta` (true effect) and `sigma`, respectively. 
This allows us to estimate the population-level parameters `mu` and `tau` by pooling from all the observations, while still allowing for individual variation amongst the schools using the group-level `theta` parameters.
This is written in `numpyro` using:

In [3]:
def eight_schools(J, sigma, y=None):
    mu = numpyro.sample('mu', dist.Normal(0, 5))
    tau = numpyro.sample('tau', dist.HalfCauchy(5))
    with numpyro.plate('J', J):
        theta = numpyro.sample('theta', dist.Normal(mu, tau))
        numpyro.sample('obs', dist.Normal(theta, sigma), obs=y)

Let us infer the values of the unknown parameters in our model by running MCMC using the No-U-Turn Sampler (NUTS). Note the usage of the extra_fields argument in MCMC.run. By default, we only collect samples from the target (posterior) distribution when we run inference using MCMC. However, collecting additional fields like potential energy or the acceptance probability of a sample can be easily achieved by using the extra_fields argument. For a list of possible fields that can be collected, see the `HMCState` object. In this example, we will additionally collect the `potential_energy` for each sample.

In [4]:
kernel = HMC(eight_schools)
mcmc = MCMC(kernel, num_warmup=500, num_samples=1000)
rng_key = random.PRNGKey(0)
mcmc.run(rng_key, J, sigma, y=y, extra_fields=('potential_energy',))
mcmc.print_summary()

rng_key.ndim == 1, setting init_state to hmc_init_fn()
hmc_init_fn in HMC.init()
Init kernel in hmc()


  0%|          | 0/1500 [00:00<?, ?it/s]

hmc_state=HMCState(i=0, z={'mu': DeviceArray(1.5965929, dtype=float32), 'tau': DeviceArray(1.9340096, dtype=float32), 'theta': DeviceArray([-1.3776793 ,  0.82720995,  0.3365407 ,  1.180201  ,
              1.919496  , -1.1188812 , -0.68791294,  1.6057692 ],            dtype=float32)}, z_grad={'mu': DeviceArray(0.2747002, dtype=float32), 'tau': DeviceArray(7.814204, dtype=float32), 'theta': DeviceArray([-0.19272894, -0.0878078 , -0.01330139, -0.05679999,
              0.04279175, -0.07426409, -0.23462468, -0.03188918],            dtype=float32)}, potential_energy=DeviceArray(58.490784, dtype=float32), energy=DeviceArray(6.9548197, dtype=float32), num_steps=0, accept_prob=0.0, mean_accept_prob=0.0, diverging=False, adapt_state=HMCAdaptState(step_size=array(1., dtype=float32), inverse_mass_matrix=DeviceArray([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32), mass_matrix_sqrt=DeviceArray([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32), ss_state=(0.0, 0.0, 0.0, 0, DeviceArra

sample: 100%|██████████| 1500/1500 [00:02<00:00, 622.73it/s, 38 steps of size 1.65e-01. acc. prob=0.79]



                mean       std    median      5.0%     95.0%     n_eff     r_hat
        mu      4.31      3.13      4.22     -0.89      8.82   1495.10      1.00
       tau      4.50      3.54      3.54      0.97      8.88     79.52      1.00
  theta[0]      6.65      5.72      6.04     -2.75     15.09    494.97      1.00
  theta[1]      4.80      4.90      4.71     -4.33     11.91    724.20      1.00
  theta[2]      3.52      5.22      3.90     -4.23     12.86    531.26      1.00
  theta[3]      4.72      4.81      4.95     -2.78     12.60    666.05      1.00
  theta[4]      3.45      4.64      3.62     -4.34     10.90    411.74      1.00
  theta[5]      3.63      4.96      3.79     -3.99     11.52    595.29      1.00
  theta[6]      6.77      5.20      6.49     -2.13     14.62    354.23      1.00
  theta[7]      4.95      5.71      4.82     -4.41     13.42    634.33      1.00

Number of divergences: 14


In [6]:
pe = mcmc.get_extra_fields()['potential_energy']
print('Expected log joint density: {:.2f}'.format(np.mean(-pe)))

Expected log joint density: -55.26


The values above 1 for the split Gelman Rubin diagnostic `r_hat` indicates that the chain has not fully converged. The low value for the effective sample size `n_eff`, particularly for `tau`, and the number of divergent transitions looks problematic. 
Fortunately, this is a common pathology that can be rectified by using a non-centered paramaterization for `tau` in our model. This is straightforward to do in `numpyro` by using a `TransformedDistribution` instance together with a "reparameterization effect handler". Let us rewrite the same model but instead of sampling `theta` from a Normal(`mu`, `tau`), we will instead sample it from a base Normal(0, 1) distribution that is transformed using an `AffineTransform`. 
Note that by doing so, `nunmpyro` runs HMC by generating samples `theta_base` for the base Normal(0, 1) distribution instead. We see that the resulting chain does not suffer from the same pathology — the Gelman Rubin diagnostic is 1 for all the parameters and the effective sample size looks quite good!

In [7]:
def eight_schools_noncentered(J, sigma, y=None):
    mu = numpyro.sample('mu', dist.Normal(0, 5))
    tau = numpyro.sample('tau', dist.HalfCauchy(5))
    with numpyro.plate('J', J):
        with numpyro.handlers.reparam(config={'theta': TransformReparam()}):
            theta = numpyro.sample(
                'theta',
                dist.TransformedDistribution(dist.Normal(0., 1.),
                                             dist.transforms.AffineTransform(mu, tau)))
        numpyro.sample('obs', dist.Normal(theta, sigma), obs=y)

nuts_kernel = NUTS(eight_schools_noncentered)
mcmc = MCMC(nuts_kernel, num_warmup=500, num_samples=1000)
rng_key = random.PRNGKey(0)
mcmc.run(rng_key, J, sigma, y=y, extra_fields=('potential_energy',))
mcmc.print_summary(exclude_deterministic=False)
pe = mcmc.get_extra_fields()['potential_energy']
print('Expected log joint density: {:.2f}'.format(np.mean(-pe)))

sample: 100%|██████████| 1500/1500 [00:04<00:00, 301.80it/s, 15 steps of size 3.87e-01. acc. prob=0.93]



                   mean       std    median      5.0%     95.0%     n_eff     r_hat
           mu      4.14      3.49      4.21     -1.43     10.12    659.31      1.00
          tau      3.85      3.19      2.98      0.00      8.34    558.82      1.00
     theta[0]      6.49      5.58      5.98     -1.49     15.95    848.21      1.00
     theta[1]      4.80      4.93      4.73     -3.35     12.51   1203.26      1.00
     theta[2]      3.68      5.46      3.94     -5.38     11.61    873.89      1.00
     theta[3]      4.70      4.64      4.74     -1.86     12.49   1187.36      1.00
     theta[4]      3.30      4.86      3.72     -4.00     11.15    719.72      1.00
     theta[5]      3.75      4.90      3.90     -3.37     12.29    619.41      1.00
     theta[6]      6.48      5.28      5.98     -1.60     14.33   1009.13      1.00
     theta[7]      5.02      5.47      4.82     -3.39     13.61    990.37      1.00
theta_base[0]      0.41      0.99      0.44     -1.12      2.08    714.07  

Now, assume that we have a new school for which we have not observed any test scores, but we would like to generate predictions. `numpyro` provides a `Predictive` class for such a purpose. Note that in the absence of any observed data, we simply use the population-level parameters to generate predictions. The `Predictive` utility conditions the unobserved `mu` and `tau` sites to values drawn from the posterior distribution from our last MCMC run, and runs the model forward to generate predictions.

In [8]:
def new_school():
    mu = numpyro.sample('mu', dist.Normal(0, 5))
    tau = numpyro.sample('tau', dist.HalfCauchy(5))
    return numpyro.sample('obs', dist.Normal(mu, tau))

predictive = Predictive(new_school, mcmc.get_samples())
samples_predictive = predictive(random.PRNGKey(1))
print(np.mean(samples_predictive['obs']))

4.0959787
